In [2]:
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
model_id, model_version = "meta-textgeneration-llama-2-7b-f", "*"

from sagemaker.jumpstart.model import JumpStartModel

endpoint_name = "<endpoint_name>"
# If endpoint is not already deployed and available, uncomment and run the code below

# my_model = JumpStartModel(model_id=model_id)
# predictor = my_model.deploy()
# endpoint_name = predictor.endpoint_name

In [ ]:
!pip3 install new_dist/*.whl --upgrade --upgrade-strategy only-if-needed --force-reinstall;

In [ ]:
from amazon_fmeval.data_loaders.data_config import DataConfig
from amazon_fmeval.model_runners.sm_jumpstart_model_runner import JumpStartModelRunner
from amazon_fmeval.eval_algo_mapping import get_eval_algorithm
from amazon_fmeval.constants import MIME_TYPE_JSONLINES
from amazon_fmeval.eval_algorithms.factual_knowledge import FactualKnowledge, FactualKnowledgeConfig

In [ ]:
# We create an instance of DataConfig which tells us about 
# the data that should be used for an evaluation. 
# This step is only necessary for custom datasets. 
config = DataConfig(
    dataset_name="tiny_dataset",
    dataset_uri="tiny_dataset.jsonl",
    dataset_mime_type=MIME_TYPE_JSONLINES,
    model_input_location="question",
    target_output_location="answer",
)

In [ ]:
# We also a create a JumpStartModelRunner which can perform invocation on 
# JumpStart models, and represent the model being evaluated.

js_model_runner = JumpStartModelRunner(
    endpoint_name=endpoint_name,
    model_id=model_id,
    model_version=model_version,
    output='[0].generation',
    content_template='{"inputs":"$prompt"}',
    custom_attributes="accept_eula=true"
)

In [ ]:
eval_algo = FactualKnowledge(FactualKnowledgeConfig(target_output_delimiter="<OR>"))
eval_output = eval_algo.evaluate(model=js_model_runner, dataset_config=config, prompt_template="$feature")